参考: https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

# データの準備

In [ ]:
from io import open
import os
import unicodedata
import string

all_letters=string.ascii_letters+" .,;'-"
n_letters=len(all_letters)+1 # EOSマーカー含む

def findFiles(path): # ファイルを検索する関数
    return glob.glob(path)

# UnicodeをASCIIに変換する
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# ファイルを読み込んで行に分割する関数
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

category_lines = {} # カテゴリーの辞書、行の内容
all_categories = [] # カテゴリーのリスト

for filename in findFiles('../data/names/*.txt'): # ファイル
    category = os.path.splitext(os.path.basename(filename))[0] # カテゴリー
    all_categories.append(category) # カテゴリーのリストに追加
    lines = readLines(filename) # 行を読む
    category_lines[category] = lines # 

n_categories = len(all_categories) # カテゴリー数

if n_categories == 0: # カテゴリーの数が0個の場合
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# ネットワークを構築する

In [ ]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(RNN,self).__init__()
        self.hidden_size=hidden_size
        
        self.i2h=nn.Linear(n_categories+input_size+hidden_size,hidden_size)
        self.i2o=nn.Linear(n_categories+input_size+hidden_size,output_size)
        self.o2o=nn.Linear(hidden_size+output_size,output_size)
        self.dropout=nn.Dropout(0.1)
        self.softmax=nn.LogSoftmax(dim=1)
        
    def forward(self,category,input,hidden):
        input_combined=torch.cat((category,input,hidden),1)
        hidden=self.i2h(input_combined)
        output=self.i2o(input_combined)
        output_combined=torch.cat((hidden,output),1)
        output=self.o2o(output_combined)
        output=self.dropout(output)
        output=self.softmax(output)
        
        return output,hidden
    
    def initHidden(self):
        return torch.zeros(1,self.hidden_size)

# 学習

## 学習の準備

In [ ]:
import random

# リストからランダムに取り出す
def randomChoice(l):
    return l[random.randint(0,len(l)-1)]

# ランダムなカテゴリを取得し、そのカテゴリからランダムな行を取得
def randomTrainingPair():
    category=randomChoice(all_categories)
    line=randomChoice(category_lines[category])
    
    return category,line

In [ ]:
# カテゴリのone-hotベクトル
def categoryTensor(category):
    li=all_categories.index(category)
    tensor=torch.zeros(1,n_categories)
    tensor[0][li]=1
    
    return tensor

# 入力の最初から最後の文字(EOSを除く)のone-hot行列
def inputTensor(line):
    tensor=torch.zeros(len(line),1,n_letters)
    
    for li in range(len(line)):
        letter=line[li]
        tensor[li][0][all_letters.find(letter)]=1
        
    return tensor

# 2文字目からEOSまでのLongTensor
def targetTensor(line):
    letter_indexes=[all_letters.find(line[li]) for li in range(1,len(line))]
    letter_indexes.append(n_letters-1) # EOS
    
    return torch.LongTensor(letter_indexes)

In [ ]:
# カテゴリ、入力、ランダムなカテゴリからの目的のテンソルと行のペア
def randomTrainingExample():
    category,line=randomTrainingPair()
    category_tensor=categoryTensor(category)
    input_line_tensor=inputTensor(line)
    target_line_tensor=targetTensor(line)
    
    return category_tensor,input_line_tensor,target_line_tensor

## ネットワークの学習

In [ ]:
criterion=nn.NLLoss()
learning_rate=0.0005

def train(category_tensor,input_line_tensor,target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden=rnn.initHidden()
    rnn.zero_grad()
    loss=0
    
    for i in range(input_line_tensor.size(0)):
        output,hidden=rnn(category_tensor,input_line_tensor[i],hidden)
        l=criterion(output,target_line_tensor[i])
        loss+=l
        
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(p.grad.data,alpha=-learning_rate)
        
    return output,loss.item()/input_line_tensor.size(0)

In [ ]:
import time
import math

def timeSince(since):
    now=time.time()
    s=now-since
    m=math.floor(s/60)
    s-=m*60
    
    return '%dm %ds'%(m, s)

In [ ]:
rnn=RNN(n_letters,128,n_letters)

n_iters=100000
print_every=5000
plot_every=500
all_losses=[]
total_loss=0 # plot_every回ごとに反復
start=time.time()

for iter in range(1,n_iters+1):
    output,loss=train(*randomTrainingExample())
    total_loss+=loss
    
    if iter%print_every==0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))
        
    if iter%plot_every==0:
        all_losses.append(total_loss/plot_every)
        total_loss=0

# ロスの表示

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

# ネットワークのサンプリング

In [ ]:
max_length=20

# カテゴリと最初の文字からサンプリングする
def sample(category,start_letters="A"):
    with torch.no_grad(): # サンプリングではhistoryをtrackする必要がない
        category_tensor=categoryTensor(category)
        input=inputTensor(start_letter)
        hidden=rnn.initHidden()
        output_name=start_letter
        
        for i in range(max_length):
            output,hidden=rnn(category_tensor,input[0],hidden)
            topv,topi=output.topk(1)
            topi=topi[0][0]
            
            if topi==n_letters-1:
                break
            else:
                letter=all_letters[topi]
                output_name+=letter
                
            input=inputTensor(letter)
            
        return output_name
    
# 一つのカテゴリから複数のサンプルと最初の文字を取得する
def samples(category,start_letters="ABC"):
    for start_letter in start_letters:
        print(sample(category,start_letter))
        
samples("Russian","RUS")
samples("German","GER")
samples("Spanish","SPA")
samples("Chinese","CHI")